<a href="https://colab.research.google.com/github/miller00315/crew_ai_studies/blob/main/multiagent_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install crewai crewai_tools langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.1/252.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.5/548.5 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.4/211.4 kB 16.7 MB/s eta

In [11]:
from crewai import LLM
import os

google_llm = LLM(
              model='gemini/gemini-1.5-flash',
              api_key='')

In [2]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [3]:
from crewai import Agent, Task, Crew

* By not setting allow_delegation=False, allow_delegation takes its default value of being True.
* This means the agent can delegate its work to another agent which is better suited to do a particular task.

In [12]:
support_agent = Agent(
    llm=google_llm,
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

In [13]:
support_quality_assurance_agent = Agent(
  llm=google_llm,
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

* Role Playing: Both agents have been given a role, goal and backstory.
* Focus: Both agents have been prompted to get into the character of the roles they are playing.
* Cooperation: Support Quality Assurance Agent can delegate work back to the Support Agent, allowing for these agents to work together.

# Tools

In [6]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("website_url")
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/selenium_scraping_tool/selenium_scraping_tool.py:26: PydanticDeprecatedSince20: Pydantic V1 style `@validator`

In [7]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

Different Ways to Give Agents Tools
* Agent Level: The Agent can use the Tool(s) on any Task it performs.
* Task Level: The Agent will only use the Tool(s) when performing that specific Task.

Note: Task Tools override the Agent Tools.

Creating Tasks
* You are passing the Tool on the Task Level.

In [24]:
support_agent = Agent(
    llm=google_llm,
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

In [25]:
support_quality_assurance_agent = Agent(
    llm=google_llm,
    role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at crewAI (https://crewai.com) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

In [26]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

In [27]:
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://docs.crewai.com/how-to/Creating-a-Crew-and-kick-it-off/"
)

In [28]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [29]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)

In [37]:
import os

crew = Crew(
    agents=[support_agent, support_quality_assurance_agent],
    tasks=[inquiry_resolution, quality_assurance_review],
    verbose=True
  )

In [38]:
inputs = {
    "customer": "DeepLearningAI",
    "person": "Andrew Ng",
    "inquiry": "I need help with setting up a Crew "
               "and kicking it off, specifically "
               "how can I add memory to my crew? "
               "Can you provide guidance?"
}

result = crew.kickoff(inputs=inputs)

 
[2025-03-11 02:00:09][🚀 CREW 'CREW' STARTED, 11244F35-E330-443D-98D9-8E80926D8FEB]: 2025-03-11 02:00:09.373466
 
[2025-03-11 02:00:09][📋 TASK STARTED: DEEPLEARNINGAI JUST REACHED OUT WITH A SUPER IMPORTANT ASK:
I NEED HELP WITH SETTING UP A CREW AND KICKING IT OFF, SPECIFICALLY HOW CAN I ADD MEMORY TO MY CREW? CAN YOU PROVIDE GUIDANCE?

ANDREW NG FROM DEEPLEARNINGAI IS THE ONE THAT REACHED OUT. MAKE SURE TO USE EVERYTHING YOU KNOW TO PROVIDE THE BEST SUPPORT POSSIBLE.YOU MUST STRIVE TO PROVIDE A COMPLETE AND ACCURATE RESPONSE TO THE CUSTOMER'S INQUIRY.]: 2025-03-11 02:00:09.400170
 
[2025-03-11 02:00:09][🤖 AGENT 'SENIOR SUPPORT REPRESENTATIVE' STARTED TASK]: 2025-03-11 02:00:09.402248
# Agent: Senior Support Representative
## Task: DeepLearningAI just reached out with a super important ask:
I need help with setting up a Crew and kicking it off, specifically how can I add memory to my crew? Can you provide guidance?

Andrew Ng from DeepLearningAI is the one that reached out. Make sure

In [40]:
from IPython.display import Markdown
Markdown(str(result))

Subject: Re: Setting up your Crew and Adding Memory

Hi Andrew,

Thanks for reaching out!  We're excited to help you get your Crew up and running smoothly.  I understand you're looking to add memory to your Crew, and to give you the best possible assistance, let's clarify what you mean by "adding memory" in the context of your Crew's operation.

Your question is a great one, and it highlights an important aspect of CrewAI that isn't explicitly covered in all of our documentation.  While our docs explain Crew creation and agent interaction, the nuances of persistent storage and internal memory management require a more tailored approach.

To help me tailor the perfect solution for you, could you please provide some more details?  Let's break down the possibilities:


**1. Persistent Storage (Remembering between Runs):**

*   **Scenario:** Your Crew needs to remember information from one run to the next.  For example, perhaps your Crew is analyzing data streams, and it needs to maintain a running total or a historical record of events.

*   **Solution:**  This requires persistent storage outside of the Crew's immediate memory.  We can integrate with various solutions depending on your needs and preferences.  This could include:
    *   **Databases:**  SQL databases (like PostgreSQL or MySQL) or NoSQL databases (like MongoDB or Cassandra) provide robust and scalable solutions for storing and retrieving information.  We can help you choose the best database based on your data structure and access patterns.
    *   **Filesystems:**  For simpler scenarios, storing data in files (e.g., JSON, CSV, or custom formats) could suffice. We can help you design the file structure for optimal data access and management.
    *   **Cloud Storage:** Services like AWS S3 or Google Cloud Storage offer scalable and cost-effective storage options, especially for large datasets.

*   **Implementation Considerations:** We'll need to discuss your data structure, expected data volume, access frequency, and any specific requirements for data integrity and security.


**2. Increasing Agent Working Memory:**

*   **Scenario:** Individual agents within your Crew are running out of memory to process their assigned tasks.  This might manifest as performance issues, crashes, or incorrect results.

*   **Solution:**  This requires adjusting the parameters within your agent's code, depending on the programming language and framework you're using.  Increasing the available memory might involve:
    *   **Python:**  Optimizing data structures and using memory-efficient libraries.  We can help you identify bottlenecks and optimize memory usage in your code.
    *   **Other Languages:**  The approach will depend on the language, but it generally involves similar principles of optimizing data structures and algorithms.

*   **Implementation Considerations:** To help me advise you effectively, please provide details about the programming languages and frameworks used, the agent's code, and the nature of the data it processes.  Error messages or performance metrics will also be incredibly helpful.


**3. Shared Memory for Inter-Agent Communication:**

*   **Scenario:** Your agents need to share information and coordinate their actions efficiently.  Passing large amounts of data between agents repeatedly can impact performance.

*   **Solution:**  Implementing a shared memory system (using appropriate data structures and locking mechanisms) allows agents to access and modify information in a controlled manner.  This could involve:
    *   **Queues:** Message queues (e.g., RabbitMQ, Kafka) enable asynchronous communication between agents, reducing contention and improving performance.
    *   **Shared Memory Data Structures:**  In languages like Python, using `multiprocessing.shared_memory` or similar mechanisms can enable efficient shared memory access.

*   **Implementation Considerations:**  We need to understand the communication pattern between your agents, the type and size of data being shared, and the level of concurrency involved.  This will determine the most suitable approach for shared memory management.



To give you the most effective solution, please tell me more about:

*   The specific task your Crew is designed to perform.
*   The architecture of your Crew (number of agents, their roles, and interactions).
*   The programming language(s) and frameworks used in your agent code.
*   The size and type of data your Crew is processing.
*   Any specific performance issues or limitations you are experiencing.


Once we have a clear picture of your requirements, we can collaborate on the best approach for adding the necessary "memory" to your Crew. We look forward to helping you achieve your goals!

Best regards,

The CrewAI Support Team